In [86]:
import numpy as np
import sklearn

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [87]:
# Importamos los datos solo con las características más importantes

X_reduc2=np.load("./datos_mfcc_pca99_X_reducido2.npy")
y_reduc=np.load("./datos_mfcc_pca99_y_reducido.npy")

In [88]:
print(X_reduc2.shape)

(360, 44)


In [89]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc2, y_reduc, test_size=0.3, shuffle=True, random_state=0)


In [100]:
# Intentamos hacer bagging con árboles de decisión. Lo intentamos primero con los parámetros por defecto. 
# Por defecto, la implementación de bagging para problemas de clasificación usa árboles de decisión. Sin embargo, vamos a usar
# los árboles con los parámetros adecuados que ya habíamos calculado en el notebook de Árboles.

arbol = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)

bagging_trees = BaggingClassifier(base_estimator=arbol, random_state=0)
bagging_trees.fit(Xtr,ytr)

preds_test = bagging_trees.predict(Xts)
print('Res bagging_trees:', metrics.classification_report(yts, preds_test))

Res bagging_trees:               precision    recall  f1-score   support

           0       0.32      0.35      0.33        17
           1       0.33      0.25      0.29        20
           2       0.63      0.60      0.62        20
           3       0.81      0.59      0.68        22
           4       0.56      0.56      0.56        18
           5       0.19      0.36      0.25        11

    accuracy                           0.46       108
   macro avg       0.47      0.45      0.45       108
weighted avg       0.51      0.46      0.48       108



In [101]:
# Mostramos el acierto en entrenamiento y en test del árbol final con el que nos hemos quedado.

print("OA train %0.2f" %bagging_trees.score(Xtr, ytr))
print("OA test %0.2f" %bagging_trees.score(Xts, yts))

OA train 0.76
OA test 0.46


Observamos que esta predicción ya mejora un pelín respecto a la obteníamos con un sólo árbol, pero tampoco demasiado. 
En test solo hemos mejorado un 0.01.

In [102]:
# Veamos cuáles son los parámetros por defecto. Quizás hemos agregado pocos árboles ya que el rendimiento no es muy bueno.

bagging_trees.get_params()

{'base_estimator__ccp_alpha': 0.0,
 'base_estimator__class_weight': None,
 'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 4,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_impurity_split': None,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__random_state': 0,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0),
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

En efecto, vemos como solo hemos combinado 10 árboles de decisión. Quizás son muy pocos. Probamos a cambiar dicho parámetro (`n_estimators`) para ver si mejora la cosa.

In [107]:
# Intentamos hacer bagging con árboles de decisión. Lo intentamos cambiando los parámetros por defecto. 

T=1000

bagging_trees2 = BaggingClassifier(base_estimator=arbol,n_estimators=T,random_state=0)
bagging_trees2.fit(Xtr,ytr)

preds_test2 = bagging_trees2.predict(Xts)
print('Res bagging_trees:', metrics.classification_report(yts, preds_test2))

Res bagging_trees:               precision    recall  f1-score   support

           0       0.35      0.35      0.35        17
           1       0.43      0.45      0.44        20
           2       0.60      0.60      0.60        20
           3       0.79      0.50      0.61        22
           4       0.58      0.61      0.59        18
           5       0.41      0.64      0.50        11

    accuracy                           0.52       108
   macro avg       0.53      0.53      0.52       108
weighted avg       0.54      0.52      0.52       108



In [108]:
# Mostramos el acierto en entrenamiento y en test del árbol final con el que nos hemos quedado.

print("OA train %0.2f" % bagging_trees2.score(Xtr, ytr))
print("OA test %0.2f" % bagging_trees2.score(Xts, yts))

OA train 0.83
OA test 0.52


Obsevarmos que al agregar 1000 árboles de decisión, los resultados en entrenamiento y test mejoran un poco (más en entrenamiento que en test en proporción). Sin embargo, el acierto en test que es el que nos interesa más, no ha subido gran cosa.

Intenamos hacer bagging con clasificadores del tipo SVC para ver si mejora el rendimiento. Usaremos el clasificador con los parámetros óptimos que ya calculamos en el notebook de SVM. 

In [84]:
# Definimos el clasificador con el que queremos hacer bagging.

svc = svm.SVC(C=4.6415888336127775, gamma=3.5938136638046257e-06)

N=500

bagging_svc = BaggingClassifier(base_estimator=svc,n_estimators=N,random_state=0)
bagging_svc.fit(Xtr,ytr)

preds_test3 = bagging_svc.predict(Xts)
print('Res bagging_svc:', metrics.classification_report(yts, preds_test3))

Res bagging_svc:               precision    recall  f1-score   support

           0       0.60      0.35      0.44        17
           1       0.63      0.60      0.62        20
           2       0.69      0.45      0.55        20
           3       0.67      0.73      0.70        22
           4       0.62      0.56      0.59        18
           5       0.31      0.73      0.43        11

    accuracy                           0.56       108
   macro avg       0.59      0.57      0.55       108
weighted avg       0.61      0.56      0.57       108



In [85]:
# Mostramos el acierto en entrenamiento y en test del árbol final con el que nos hemos quedado.

print("OA train %0.2f" % bagging_svc.score(Xtr, ytr))
print("OA test %0.2f" % bagging_svc.score(Xts, yts))

OA train 0.71
OA test 0.56


Vemos como no han mejorado los aciertos respecto a usar un solo clasificador el tipo SVC. No merece la pena hacer bagging con máquinas de vectores soporte. Esto se puede deber a que sean un método más estable, y con métodos estables no funciona muy bien el bagging.